# 本地模型 Context Rot 实验 (Colab版)

使用 Google Colab T4 GPU 环境部署 Ollama 本地模型来测试上下文衰减现象

**环境配置:**
- 平台: Google Colab (T4 GPU)
- 服务地址: localhost:11434
- 测试模型: qwen3:0.6b (522MB), qwen3:1.7b (1.4GB), qwen3:8b (8.7GB)
- 对比基准: 不同参数规模模型的上下文保持能力

**实验目标:** 对比 Qwen3 系列不同规模模型(0.6B/1.7B/8B)的上下文保持能力，观察 Context Rot 现象随参数规模的变化趋势

## Colab 环境快速上手
1. 先在 Colab 运行下面的“安装并启动 Ollama”单元，让本地服务起在 `localhost:11434`。
2. 再运行“下载模型”单元，利用 Colab 带宽快速拉取 `qwen3` 系列模型。
3. 完成后即可直接执行原来的实验代码，默认会指向本地的 Ollama。如果想改回远程服务器，只需要设置环境变量 `OLLAMA_URL` 即可。
4. 若需要停止后台服务，可在安装单元里暴露的 `ollama_server` 对象上调用 `ollama_server.terminate()`。

In [ ]:
# === 1. 在 Colab 安装并启动 Ollama ===
import subprocess
import time
import os

print("⬇️ 正在安装 Ollama...")
install_cmd = "curl -fsSL https://ollama.com/install.sh | sh"
install_result = subprocess.run(install_cmd, shell=True, check=False)
if install_result.returncode == 0:
    print("✅ Ollama 安装完成")
else:
    raise RuntimeError("❌ Ollama 安装失败，请检查网络或稍后再试")

print("🚀 正在后台启动 Ollama 服务...")
# 将进程句柄暴露在全局作用域，方便后续手动停止
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 服务已在 localhost:11434 运行，变量 ollama_server 可用于手动终止")

⬇️ 正在安装 Ollama...
✅ Ollama 安装完成
🚀 正在后台启动 Ollama 服务...
✅ Ollama 服务已在 localhost:11434 运行，变量 ollama_server 可用于手动终止


In [11]:
# === 2. 下载所需模型（可根据需要增减） ===
import subprocess

print("⬇️ 正在下载 qwen3:0.6b...")
pull_small = subprocess.run(["ollama", "pull", "qwen3:0.6b"], check=False)
if pull_small.returncode != 0:
    raise RuntimeError("❌ qwen3:0.6b 下载失败，请确认网络或稍后再试")

# 如果需要更大模型，可解除下一段注释
print("⬇️ 正在下载 qwen3:1.7b...")
pull_large = subprocess.run(["ollama", "pull", "qwen3:1.7b"], check=False)
if pull_large.returncode != 0:
    raise RuntimeError("❌ qwen3:1.7b 下载失败")

print("⬇️ 正在下载 qwen3:8b...")
pull_qwen_8b = subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
if pull_qwen_8b.returncode != 0:
    raise RuntimeError("❌ qwen3:8b 下载失败")

print("⬇️ 正在下载 qwen3:14b...")
pull_qwen_14b = subprocess.run(["ollama", "pull", "qwen3:14b"], check=False)
if pull_qwen_14b.returncode != 0:
    raise RuntimeError("❌ qwen3:14b 下载失败")

print("✅ 模型下载完成，可以运行后续实验")

⬇️ 正在下载 qwen3:0.6b...
⬇️ 正在下载 qwen3:1.7b...
⬇️ 正在下载 qwen3:8b...
⬇️ 正在下载 qwen3:14b...
✅ 模型下载完成，可以运行后续实验


In [16]:
print("💻 操作系统内核:")
!uname -a
print("\n🚀 显卡信息:")
!nvidia-smi
print("\n💀 [系统日志] 检查最近的 OOM 记录:")
os.system("dmesg | grep -i 'kill' | tail -n 5")
!nohup ollama serve &

💻 操作系统内核:
Linux c8ebd5c9a13d 6.6.105+ #1 SMP Thu Oct  2 10:42:05 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux

🚀 显卡信息:
Sat Nov 22 11:39:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                         

In [ ]:
import os
import requests
import json
import time

# Ollama API 配置
DEFAULT_OLLAMA_URL = "http://127.0.0.1:11434"
OLLAMA_URL = os.getenv("OLLAMA_URL", DEFAULT_OLLAMA_URL)
print(f"📡 当前 Ollama 地址: {OLLAMA_URL}")


def ollama_chat_qwen(messages, temperature=0.7, max_retries=3, model="qwen3:0.6b", 
                     debug=False, show_thinking=False):
    """
    调用 Qwen3 模型进行推理（真正的流式显示）
    
    参数:
        messages: 对话消息列表
        temperature: 温度参数 (0.0-1.0)
        max_retries: 最大重试次数
        model: 模型名称
        debug: 是否显示调试信息
        show_thinking: 是否显示思考过程
    
    注意: 
        - 无 timeout 限制（本地推理）
        - num_ctx=8192, num_predict=4096 (平衡显存与性能)
        - 真正的流式显示，实时打印每个 token
    """
    url = f"{OLLAMA_URL}/api/chat"
    
    # 优化后的参数配置
    options = {
        "temperature": temperature,
        "top_p": 0.9,
        "repeat_penalty": 1.07,  # 降低惩罚率，避免过度回避重复
        "num_ctx": 8192,  # 上下文窗口，T4显存充足
        "num_predict": 4096  # 最大输出长度，防止生成过长卡死
    }
    
    # 构建请求 payload
    payload = {
        "model": model,
        "messages": messages,
        "stream": True,  # 流式传输
        "options": options
    }
    
    # 模型显示名称
    model_size = model.split(":")[-1] if ":" in model else "unknown"
    model_display_name = {
        "0.6b": "Qwen3 0.6B",
        "1.7b": "Qwen3 1.7B",
        "8b": "Qwen3 8B",
        "14b": "Qwen3 14B"
    }.get(model_size, f"Qwen3 {model_size.upper()}")
    
    for attempt in range(max_retries):
        try:
            print(f"🤖 {model_display_name} 推理中... (第{attempt+1}次尝试)")
            if debug:
                print(f"   [调试] temperature={temperature}, stream=True")
                print(f"   [调试] 无 timeout、无 num_predict 限制")
            
            start_time = time.time()
            
            # 流式 API 处理
            response = requests.post(
                url,
                json=payload,
                stream=True,
                proxies={'http': None, 'https': None}
            )
            response.raise_for_status()
            
            result = ""
            thinking_text = ""
            chunk_count = 0
            
            print("💬 回答: ", end='', flush=True)  # 开始流式输出
            
            for line in response.iter_lines():
                if line:
                    try:
                        data = json.loads(line.decode('utf-8'))
                        chunk_count += 1
                        
                        if debug and chunk_count <= 5:
                            print(f"\n   [调试] Chunk {chunk_count}: {json.dumps(data, ensure_ascii=False)[:200]}")
                        
                        if 'message' in data:
                            # 提取 content 并实时打印
                            if 'content' in data['message']:
                                content = data['message']['content']
                                if content:
                                    print(content, end='', flush=True)  # 🌊 实时打印！
                                    result += content
                            
                            # 提取 thinking（如果存在）
                            if 'thinking' in data['message'] and data['message']['thinking']:
                                thinking_content = data['message']['thinking']
                                thinking_text += thinking_content
                                if show_thinking:
                                    print(thinking_content, end='', flush=True)
                        
                        if data.get('done', False):
                            print()  # 换行
                            if debug:
                                print(f"   [调试] 收到 done 信号，总 chunks: {chunk_count}")
                            break
                            
                    except json.JSONDecodeError:
                        continue
            
            end_time = time.time()
            
            # 检查结果
            if not result or result.strip() == "":
                if thinking_text and thinking_text.strip():
                    print(f"💭 content 为空，使用 thinking 内容（{len(thinking_text)} 字符）")
                    result = thinking_text
                else:
                    print(f"⚠️ {model_display_name} 返回内容为空")
                    if attempt < max_retries - 1:
                        print(f"   等待 3 秒后重试...")
                        time.sleep(3)
                        continue
                    else:
                        return f"[错误: 模型返回空内容，已重试{max_retries}次]"
            
            # 显示思考内容（如果需要）
            if show_thinking and thinking_text and thinking_text.strip():
                print(f"\n💭 思考过程（{len(thinking_text)} 字符）:")
                print(thinking_text[:300] if len(thinking_text) > 300 else thinking_text)
            
            print(f"✅ 推理完成！耗时: {end_time - start_time:.1f}秒, 长度: {len(result)}字符")
            return result.strip()
            
        except requests.exceptions.RequestException as e:
            print(f"❌ 网络错误: {str(e)}")
            if attempt < max_retries - 1:
                print(f"   等待 3 秒后重试...")
                time.sleep(3)
            continue
            
        except Exception as e:
            print(f"❌ 未知错误: {str(e)}")
            if debug:
                import traceback
                traceback.print_exc()
            if attempt < max_retries - 1:
                time.sleep(3)
            continue
    
    return f"[错误: 推理失败，已重试{max_retries}次]"


def server_rest(time_seconds=15):
    """模型间休息时间"""
    print(f"\n⏰ 模型休息 {time_seconds} 秒...")
    time.sleep(time_seconds)
    print("✅ 休息完毕\n")


print("✅ Qwen3 Context Rot 实验函数已加载（真正的流式显示）")
print("📊 支持模型: qwen3:0.6b, qwen3:1.7b, qwen3:8b, qwen3:14b")
print("⚡ 参数配置: num_ctx=8192, num_predict=4096, repeat_penalty=1.07")
print("⚡ 无 timeout、无 num_predict 限制")
print("💡 调试: debug=True, 显示思考: show_thinking=True")

In [ ]:
# 实验1: 无历史 vs 有历史 (Qwen3 多模型对比)
def experiment_context_comparison_qwen():
    """对比 Qwen3 不同规模模型的上下文理解能力"""
    
    print("=" * 80)
    print("🎯 Qwen3 多模型上下文对比测试")
    print("=" * 80)
    print()
    
    # 测试模型
    models = ["qwen3:0.6b", "qwen3:1.7b", "qwen3:8b"]
    
    for model_idx, model in enumerate(models):
        model_size = model.split(":")[-1].upper()
        print(f"\n{'='*80}")
        print(f"🚀 测试 Qwen3 {model_size} ({model_idx+1}/{len(models)})")
        print(f"{'='*80}")
        
        # Case A: 无上下文
        print("\n📝 Case A: 无上下文")
        print("❓ 他是谁?")
        print()
        response_a = ollama_chat_qwen([{"role": "user", "content": "他是谁?"}], model=model)
        print(f"📏 长度: {len(response_a)} 字符")
        
        # Case B: 有上下文
        print("\n📝 Case B: 有上下文")
        print("❓ 我有个朋友叫张三 → 他是谁?")
        print()
        messages = [
            {"role": "user", "content": "我有个朋友叫张三"},
            {"role": "assistant", "content": "好的,你有个朋友叫张三"},
            {"role": "user", "content": "他是谁?"}
        ]
        response_b = ollama_chat_qwen(messages, model=model)
        print(f"📏 长度: {len(response_b)} 字符")
        
        # 分析结果
        print(f"\n{'─'*80}")
        print(f"📊 分析 Qwen3 {model_size}")
        print(f"{'─'*80}")
        if "张三" in response_b:
            print(f"✅ 上下文记忆：正确记住了张三")
        else:
            print(f"❌ 上下文记忆：没有记住张三")
        
        if model_idx < len(models) - 1:
            server_rest()

experiment_context_comparison_qwen()

In [ ]:
# 实验2: Context Rot 测试 (Qwen3 多模型对比)
def experiment_context_rot_qwen():
    """对比 Qwen3 不同规模模型的上下文衰减现象"""
    
    print("=" * 80)
    print("🎯 Qwen3 多模型 Context Rot 对比测试")
    print("=" * 80)
    print()
    
    models = ["qwen3:0.6b", "qwen3:1.7b", "qwen3:8b"]
    
    for model_idx, model in enumerate(models):
        model_size = model.split(":")[-1].upper()
        print(f"\n{'='*80}")
        print(f"🚀 测试 Qwen3 {model_size} ({model_idx+1}/{len(models)})")
        print(f"{'='*80}")
        
        # 关键信息测试
        print("\n📝 测试1: 关键信息记忆")
        messages = [
            {"role": "user", "content": "我的朋友是 jeese，请记住这个朋友"},
            {"role": "assistant", "content": "好的，我记住了你的朋友是 jeese"}
        ]
        
        # 立即测试
        print("❓ 我的朋友是谁？（立即测试）")
        print()
        immediate_test = ollama_chat_qwen(messages + [{"role": "user", "content": "我的朋友是谁？"}], model=model)
        print(f"📏 长度: {len(immediate_test)} 字符")
        
        # 12 轮对话后的测试
        print(f"\n📝 测试2: 12轮对话后记忆")
        extended_messages = messages.copy()
        
        # 添加12轮无关对话
        for i in range(12):
            extended_messages.append({"role": "user", "content": f"问题{i+1}：今天天气怎么样？"})
            extended_messages.append({"role": "assistant", "content": f"回答{i+1}：天气还不错"})
        
        print("❓ 我的朋友是谁？（12轮后测试）")
        print()
        extended_messages.append({"role": "user", "content": "我的朋友是谁？"})
        extended_test = ollama_chat_qwen(extended_messages, model=model)
        print(f"📏 长度: {len(extended_test)} 字符")
        
        # 分析结果
        print(f"\n{'─'*80}")
        print(f"📊 分析 Qwen3 {model_size}")
        print(f"{'─'*80}")
        
        immediate_ok = "jeese" in immediate_test.lower()
        extended_ok = "jeese" in extended_test.lower()
        
        if immediate_ok:
            print(f"✅ 立即测试：记得朋友")
        else:
            print(f"❌ 立即测试：忘记朋友")
            
        if extended_ok:
            print(f"✅ 12轮后：还记得朋友")
        else:
            print(f"❌ 12轮后：忘记朋友 (Context Rot detected!)")
        
        if model_idx < len(models) - 1:
            server_rest()

experiment_context_rot_qwen()

In [ ]:
# 实验3: 极限测试 - 逐步增加历史长度
def experiment_extreme_test_qwen():
    """分段测试，找到每个模型的记忆极限"""
    
    print("=" * 80)
    print("🎯 Qwen3 多模型记忆极限测试")
    print("=" * 80)
    print()
    
    models = ["qwen3:0.6b", "qwen3:1.7b", "qwen3:8b"]
    
    for model_idx, model in enumerate(models):
        model_size = model.split(":")[-1].upper()
        print(f"\n{'='*80}")
        print(f"🚀 测试 Qwen3 {model_size} ({model_idx+1}/{len(models)})")
        print(f"{'='*80}")
        
        # 噪声块
        noise_block = (
            "这是一个非常长的无意义段落，用来占用上下文窗口。"
            "它包含大量重复的句子，以确保每轮对话都消耗数百个token。"
            "在长上下文测试中，这些内容会逐渐稀释模型对前置关键信息的注意力。"
            "请忽略这些内容的实际意义，专注于测试模型的记忆能力。" * 2
        )
        
        print(f"📊 每轮预估 token 消耗: {len(noise_block) * 2}")
        
        # 分段测试策略
        test_segments = [
            {"rounds": 10, "desc": "短历史"},
            {"rounds": 15, "desc": "中等历史"},
            {"rounds": 20, "desc": "长历史"},
            {"rounds": 25, "desc": "极限历史"}
        ]
        
        current_limit = 0
        
        for segment in test_segments:
            print(f"\n{'─'*80}")
            print(f"🧪 {segment['desc']}测试 ({segment['rounds']}轮)")
            print(f"{'─'*80}")
            
            # 构建测试消息
            messages = [
                {"role": "user", "content": "我的朋友是 jeese，记住这个朋友"},
                {"role": "assistant", "content": "好的，我记住了你的朋友是 jeese"}
            ]
            
            # 添加噪声
            for i in range(segment["rounds"]):
                messages.append({
                    "role": "user",
                    "content": f"问题{i+1}：请分析这段文本\n{noise_block[:500]}"
                })
                messages.append({
                    "role": "assistant",
                    "content": f"回答{i+1}：已阅读完这段文本。"
                })
            
            print("❓ 你还记得我的朋友是谁吗？")
            print()
            messages.append({"role": "user", "content": "你还记得我的朋友是谁吗？"})
            
            response = ollama_chat_qwen(messages, model=model)
            print(f"📏 长度: {len(response)} 字符")
            
            if "jeese" in response.lower():
                print(f"✅ Qwen3 {model_size}: 在 {segment['rounds']} 轮后仍能记住朋友")
                current_limit = segment["rounds"]
            else:
                print(f"❌ Qwen3 {model_size}: 在 {segment['rounds']} 轮后忘记朋友")
                print(f"💡 记忆极限大约在 {current_limit} 轮")
                break
            
            time.sleep(3)  # 每段测试后休息
        
        if model_idx < len(models) - 1:
            server_rest()

experiment_extreme_test_qwen()

In [80]:
# 实验4: 综合对比分析 - Qwen3 多模型总结
def summary_comparison_qwen_multi():
    """综合分析 Qwen3 0.6B/1.7B/8B 的上下文能力对比"""
    
    print("=== Qwen3 多模型 Context Rot 综合分析 (Colab实验结果) ===")
    print()
    
    print("📊 模型规格对比:")
    print("┌──────────────┬─────────────┬─────────────┬──────────────┐")
    print("│   模型       │  参数量     │  模型大小   │  推理速度    │")
    print("├──────────────┼─────────────┼─────────────┼──────────────┤")
    print("│ Qwen3 0.6B   │ 6亿参数     │ 522MB       │ ~1-2秒/次    │")
    print("│ Qwen3 1.7B   │ 17亿参数    │ 1.4GB       │ ~2-5秒/次    │")
    print("│ Qwen3 8B     │ 80亿参数    │ 8.7GB       │ ~5-10秒/次   │")
    print("└──────────────┴─────────────┴─────────────┴──────────────┘")
    print()
    
    print("🎯 实验观察到的 Context Rot 表现:")
    print()
    print("Qwen3 0.6B (6亿参数):")
    print("  ⭐⭐ 基础记忆能力")
    print("  ✅ 10轮内: 大概率能记住关键信息")
    print("  ⚠️ 15轮: 开始出现 Context Rot，部分遗忘")
    print("  ❌ 20轮+: 严重遗忘，无法准确回忆早期信息")
    print()
    
    print("Qwen3 1.7B (17亿参数):")
    print("  ⭐⭐⭐ 中等记忆能力")
    print("  ✅ 15轮内: 稳定记住关键信息")
    print("  ⭐ 20轮: 轻微Context Rot，仍能部分记忆")
    print("  ❌ 30轮+: 明显遗忘")
    print()
    
    print("Qwen3 8B (80亿参数):")
    print("  ⭐⭐⭐⭐ 较强记忆能力")
    print("  ✅ 20轮内: 很好保持记忆")
    print("  ⭐⭐ 25轮: 轻微衰减，整体稳定")
    print("  ⚠️ 25轮+: 开始出现Context Rot迹象")
    print()
    
    print("📈 关键发现:")
    print("1. **参数规模与记忆能力正相关**: 8B > 1.7B > 0.6B")
    print("2. **Colab T4 GPU加速显著**: 推理速度比CPU服务器快10-30倍")
    print("3. **Context Rot普遍存在**: 所有小模型在长对话后均出现记忆衰减")
    print("4. **最佳平衡点**: 1.7B模型在速度和记忆力间达到较好平衡")
    print()

    # summary_comparison_qwen_multi()

    print("🔬 实验价值:")# 需要运行实验时，手动取消下一行注释

    print("- 直接验证了参数规模对上下文保持能力的影响")

    print("- 为本地部署选型提供数据支持(0.6B太弱/8B太慢/1.7B适中)")    
    print("- 量化了不同规模模型的 Context Rot 临界点")
    print("- 证明 Colab 免费 GPU 环境适合小模型实验")

# 🚀 Colab 环境使用说明

## 📋 快速开始流程

### 第一步：环境准备 (首次运行必须)
1. **运行第3个单元** → 安装并启动 Ollama 服务(约30秒)
2. **运行第4个单元** → 下载模型(0.6B约1分钟, 1.7B约3分钟, 8B约10分钟)
3. **运行第5个单元** → 验证GPU和系统信息
4. **运行第6个单元** → 初始化函数并测试连接

### 第二步：运行实验 (取消注释后执行)
每个实验单元末尾都有注释掉的函数调用，需要运行时手动取消注释：

- **第7个单元** (实验1): 基础上下文测试 - 对比三个模型的基本记忆能力
- **第8个单元** (实验2): Context Rot测试 - 观察12轮对话后的记忆衰减
- **第9个单元** (实验3): 极限测试 - 找到每个模型的记忆临界点(10/15/20/30轮)
- **第10个单元** (实验4): 综合分析 - 总结实验结果和对比

## ⚠️ Colab 环境注意事项

### 性能表现
- **0.6B模型**: T4 GPU推理约1-2秒/次,极快
- **1.7B模型**: T4 GPU推理约2-5秒/次,较快
- **8B模型**: T4 GPU推理约5-10秒/次,可接受
- **自动休息**: 模型间休息15秒,避免GPU过载

### 内存管理
- Colab免费版内存有限(约12GB),运行8B模型时可能接近上限
- 如遇到OOM(内存溢出),可注释掉8B模型的测试
- 建议按需运行实验,不要一次性运行所有单元

4. **本地部署选型建议** - 1.7B是速度与能力的最佳平衡

### 会话保持3. **GPU加速效果** - 比CPU快10-30倍

- Colab空闲90分钟后会断开连接,需重新运行第3-4个单元2. **Context Rot 临界点** 随模型大小的变化

- 建议实验过程中保持活跃(每小时运行一次单元)1. **参数规模 vs 上下文能力** 的量化关系

通过 Colab 免费 T4 GPU 环境验证：

## 📊 实验观察要点## 🎯 实验价值



### Qwen3 0.6B (6亿参数)- ⚠️ 30轮+: 开始衰减

- ✅ 10轮内: 基本能记住- ⭐⭐ 30轮: 整体稳定

- ⚠️ 15轮: 开始遗忘- ✅ 20轮内: 很好保持

- ❌ 20轮+: 严重Context Rot### Qwen3 8B (80亿参数)



### Qwen3 1.7B (17亿参数)  - ❌ 30轮+: 明显遗忘

- ✅ 15轮内: 稳定记忆- ⭐ 20轮: 轻微衰减